<a href="https://colab.research.google.com/github/Charan6924/MachineLearningProjects/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import csv
f = open(train_file_path, 'r')
csv_f = csv.reader(f, delimiter='\t')
count = 0
for row in csv_f:
  print(row)
  count += 1
print(count)

In [ ]:
t = open(test_file_path, 'r')
csv_t = csv.reader(t, delimiter='\t')
count = 0
for row in csv_t:
  print(row)
  count += 1
print(count)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data = pd.read_csv(train_file_path,delimiter='\t')
test_data = pd.read_csv(test_file_path, delimiter = '\t')

train_data.columns = ["label", "message"]
test_data.columns = ["label", "message"]

train_data["label"] = train_data["label"].map({"ham": 0, "spam": 1})
test_data["label"] = test_data["label"].map({"ham": 0, "spam": 1})

X_train = train_data["message"]
y_train = train_data["label"]
X_test = test_data["message"]
y_test = test_data["label"]


In [ ]:
# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Ensure all token indices are within the range [0, num_words-1]
X_train_seq = [[token if token < 10000 else 0 for token in seq] for seq in X_train_seq]
X_test_seq = [[token if token < 10000 else 0 for token in seq] for seq in X_test_seq]

# Pad sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=100)
X_test_padded = pad_sequences(X_test_seq, maxlen=100)



In [ ]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=100),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    X_train_padded, y_train,
    epochs=100,
    validation_data=(X_test_padded, y_test),
    verbose=2
)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")


In [ ]:
print(predict_message("Win a free iPhone now! Click the link!"))
print(predict_message("Hey, are we meeting at the café today?"))
print(predict_message("Urgent! Your account will be suspended unless you verify it immediately."))
print(predict_message("Happy Birthday! Hope you have a great day!"))


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(message):
    """Predict whether a message is spam or ham."""


    sequence = tokenizer.texts_to_sequences([message])
    sequence = [[min(token, 10000-1) for token in seq] for seq in sequence]
    padded = pad_sequences(sequence, maxlen=100)

    prob = float(model.predict(padded, verbose=0)[0][0])
    label = "spam" if prob > 0.01 else "ham"

    return prob, label

pred_text = "URGENT! You have won a prize"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
